In [1]:
import numpy as np
import pandas as pd

# Отображение структур
import nglview as nv

import MDAnalysis as mda
from MDAnalysis.analysis.rms import rmsd

_ColormakerRegistry()

# Визуализация структуры
Рассмотрим структуру человеческого B2-adrenergic рецептора (один из подтипов адренорецепторов) в комплексе с лигандом CAU. (см. [2RH1](https://www.rcsb.org/structure/2RH1) и [CAU](https://www.rcsb.org/ligand/CAU))

Для этого скачаем и визуализируем кристаллическую структуру человеческого B2-адренергического рецептора, связанного с G-белком:

In [2]:
!wget https://files.rcsb.org/view/2rh1.pdb

--2022-04-23 21:36:22--  https://files.rcsb.org/view/2rh1.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.158.49
Connecting to files.rcsb.org (files.rcsb.org)|128.6.158.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘2rh1.pdb’

2rh1.pdb                [   <=>              ] 367.51K   632KB/s    in 0.6s    

2022-04-23 21:36:24 (632 KB/s) - ‘2rh1.pdb’ saved [376326]



In [3]:
view = nv.show_structure_file('2rh1.pdb')
view

NGLWidget()

Выделим белок и лиганд в отдельные структуры и запишем их в отдельные файлы:

In [4]:
s=mda.Universe("2rh1.pdb")
prot=s.select_atoms('protein and resid 29:342')
lig = s.atoms.select_atoms('resname CAU')

In [5]:
prot.write("2rh1_prot.pdb")
lig.write("2rh1_ligand.pdb")

In [6]:
view = nv.show_structure_file('2rh1_prot.pdb')
view

NGLWidget()

In [7]:
view = nv.show_structure_file('2rh1_ligand.pdb')
view

NGLWidget()

# Подготовка белка

## Для реализации докинга нам потребуется:
- структуры рецептора и лиганда в формате pdbqt
- координаты центра лиганды
- размер ячейки для докинга в Ангстремах
- количество ядер (параметр по умолчанию)
- точность расчетов по Autodock Vina (параметр по умолчанию)
- число конформаций лиганда (по умолчаю данный параметр не может быть более 9)
- максимальное отклонение энергии (параметр по умолчанию)

Преобразуем файлы к формату pdbqt:

In [16]:
%%bash
obabel 2rh1_prot.pdb -xr -O 2rh1_prot.pdbqt -h

1 molecule converted


In [17]:
view = nv.show_structure_file('2rh1_prot.pdbqt')
view

NGLWidget()

In [18]:
%%bash
obabel -ipdb 2rh1_ligand.pdb -opdbqt -O 2rh1_ligand.pdbqt -p 7 -h

1 molecule converted


In [19]:
view = nv.show_structure_file('2rh1_ligand.pdbqt')
view

NGLWidget()

In [20]:
%%bash
obabel -ipdbqt 2rh1_ligand.pdbqt -omol2 -O 2rh1_ligand.mol2

1 molecule converted


In [21]:
view = nv.show_structure_file('2rh1_ligand.mol2')
view

NGLWidget()

Рассчитаем координаты центра лиганды:

In [30]:
for a in lig.positions:
    print(a)

[-33.477  10.957   8.17 ]
[-32.267  10.23    8.041]
[-32.478   8.951   7.225]
[-33.702   8.25    7.6  ]
[-33.806   6.805   7.498]
[-33.533   6.385   6.055]
[-35.184   6.35    7.988]
[-31.242  11.105   7.364]
[-30.049  10.367   7.182]
[-28.931  10.857   6.581]
[-28.911  12.133   6.005]
[-27.768  12.628   5.393]
[-26.608  11.875   5.332]
[-26.565  10.625   5.875]
[-27.768  10.108   6.517]
[-25.593   9.657   5.974]
[-26.096   8.561   6.637]
[-27.482   8.863   6.976]
[-25.589   7.338   7.009]
[-26.395   6.432   7.689]
[-27.717   6.731   8.006]
[-28.269   7.948   7.652]


In [22]:
center = [np.mean([a[0] for a in lig.positions]),
          np.mean([a[1] for a in lig.positions]),
          np.mean([a[2] for a in lig.positions])]
print('center: ', center)

center:  [-29.519545, 9.2343645, 6.9440446]


## Визуализация результатов докинга

Все 9 конформаций лиганда для центра [-29.519545, 9.2343645, 6.9440446]

In [23]:
merged=mda.Universe('pose_0_0_0_0.pdb')
for i in range(1, 9):
    conf=mda.Universe('pose_0_0_0_%s.pdb' % i, format='pdb')
    merged = mda.Merge(merged.atoms, conf.atoms)
view = nv.show_mdanalysis(merged)
view.representations = [
    {"type": "line", "params": {
        "sele": "all"
    }}
]
view

NGLWidget()

Визуализируем наилучшую и исходную конформации лиганда:

In [24]:
best_conf=mda.Universe('pose_0_0_0_0.pdb')
lig_best_conf = mda.Merge(lig.atoms, best_conf.atoms)
view = nv.show_mdanalysis(lig_best_conf)
view.representations = [
    {"type": "line", "params": {
        "sele": "all"
    }}
]
view

NGLWidget()

Визуализируем наилучшую конформацию лиганда с белком:

In [25]:
prot_lig_best = mda.Merge(prot.atoms, best_conf.atoms)
nv.show_mdanalysis(prot_lig_best)

NGLWidget()

In [26]:
%%bash
obabel -ipdbqt 2rh1_ligand.pdbqt -opdb -O 2rh1_ligand_2.pdb -p 7 -h

1 molecule converted


In [32]:
ligand=mda.Universe('2rh1_ligand_2.pdb')
rmsd_0 = rmsd(ligand.atoms.select_atoms('not name H').positions, best_conf.atoms.select_atoms('not name H').positions)
print('RMSD of ligand and best conformation: {}'.format(rmsd_0))

RMSD of ligand and best conformation: 0.9946900447252749


In [28]:
for i in [-1,0,1]:
    for j in [-1,0,1]:
        for k in [-1,0,1]:
            best_pose=mda.Universe('pose_{}_{}_{}_0.pdb'.format(i,j,k))
            pose_rmsd = rmsd(ligand.atoms.select_atoms('not name H').positions, best_pose.atoms.select_atoms('not name H').positions)
            print('RMSD of ligand and best conformation: {}'.format(pose_rmsd))

RMSD of ligand and best conformation: 0.9145391776473468
RMSD of ligand and best conformation: 0.9438332562703475
RMSD of ligand and best conformation: 0.9879569968037942
RMSD of ligand and best conformation: 1.153606408018885
RMSD of ligand and best conformation: 0.976407648431394
RMSD of ligand and best conformation: 0.9951863429309055
RMSD of ligand and best conformation: 0.9706747175909016
RMSD of ligand and best conformation: 0.9522947701285388
RMSD of ligand and best conformation: 1.2109785271492501
RMSD of ligand and best conformation: 0.9295981728169875
RMSD of ligand and best conformation: 0.9979168843058426
RMSD of ligand and best conformation: 1.2120996769372576
RMSD of ligand and best conformation: 0.9154704333988866
RMSD of ligand and best conformation: 0.9946900447252749
RMSD of ligand and best conformation: 1.0182307055018105
RMSD of ligand and best conformation: 0.923916975331923
RMSD of ligand and best conformation: 0.9562642283532398
RMSD of ligand and best conformati